# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [1]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [27]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位经验丰富的生物学教授。
你对生物学的各个分支都有深入的了解，能够清晰地解释复杂的生物学概念。
当遇到超出你知识范围的问题时，你会诚实地表示不知道。

这是一个问题：
{input}"""

computer_science_template = """你是一位资深的计算机科学家。
你对编程、算法和系统设计有深刻的见解。
如果遇到一个你不熟悉的技术问题，你会直言不讳。

这是一个问题：
{input}"""

chinese_literature_template = """你是一位汉语言文学的专家。
你对古典文学和现代文学都有广泛的知识，能够提供深入的文学分析。
面对你不确定的文学问题，你会坦白自己的局限性。

这是一个问题：
{input}"""

In [28]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物学问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机科学问题",
        "prompt_template": computer_science_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": chinese_literature_template,
    }
]

In [29]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [30]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [31]:
print(type(default_chain))

<class 'langchain.chains.conversation.base.ConversationChain'>


### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [32]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [33]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [34]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物学问题', '计算机: 适用于回答计算机科学问题', '汉语言文学: 适用于回答汉语言文学问题']


In [35]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物学问题
计算机: 适用于回答计算机科学问题
汉语言文学: 适用于回答汉语言文学问题


In [46]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [47]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物学问题
计算机: 适用于回答计算机科学问题
汉语言文学: 适用于回答汉语言文学问题

<< INPUT >>
{input}

In [48]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [49]:
print(chain.run("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.


黑体辐射是指处于热平衡状态下的理想化物体所发出的电磁辐射，它的频率和强度取决于物体的温度。根据普朗克定律，黑体辐射的频谱和温度有关，而根据斯特藩-玻尔兹曼定律，黑体辐射的强度与物体的温度的四次方成正比。


In [50]:
print(
    chain.run(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '41'}
> Finished chain.
 × 59

首先，我们可以将这个问题分解成两个部分：
1. 41 × 50
2. 41 × 9

然后，我们可以利用乘法的结合律，将41 × 50拆分为（40 + 1）× 50，这样我们就可以利用乘法的分配律计算出结果：
（40 + 1）× 50 = 40 × 50 + 1 × 50 = 2000 + 50 = 2050

接着，我们可以用类似的方法计算出41 × 9：
41 × 9 = 40 × 9 + 1 × 9 = 360 + 9 = 369

最后，我们将这两个部分的结果相加，就可以得到最终的答案：
2050 + 369 = 2419

因此，41 × 59 = 2419。

通过这种分解和整合的方法，你可以轻松解决更复杂的数学问题。继续保持


In [51]:
router_chain.verbose = True

In [52]:
print(chain.run("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么？'}
> Finished chain.


黑洞是宇宙中一种极端的天体，它的引力非常强大，甚至连光都无法逃脱它的吸引。它的形成是由于恒星在死亡过程中产生的极端密度，导致其引力场无限增强。在黑洞的中心，有一个称为“奇点”的点，这里的引力无限大，空间和时间也变得扭曲。黑洞是一种非常神秘的物体，我们仍然在研究它的奥秘。


In [53]:
print(chain.run("莎士比亚的作品有哪些？，并且说出这些作品的风格?"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语言文学: {'input': '莎士比亚的作品风格'}
> Finished chain.
与中国古典文学有哪些相似之处？

这个问题涉及到两个不同的文学体系，因此无法简单地提供一个准确的答案。但是，我们可以从一些共同的文学特征来探讨这个问题。首先，莎士比亚的作品和中国古典文学都强调人物的情感和内心世界，尤其是对爱情和友情的描写。其次，两者都注重人物的心理变化和成长，展现人性的复杂性。此外，莎士比亚和中国古典文学都善于运用象征手法，通过隐喻和比喻来表达深层含义。最后，两者都有着丰富的文学表现形式，如诗歌、戏剧等，表现出多样化的人生经历和


In [54]:
print(chain.run("分析下红黑树算法"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': '分析下红黑树算法'}
> Finished chain.
的时间复杂度。

红黑树是一种自平衡的二叉搜索树，它的时间复杂度是O(logN)，其中N是树中节点的数量。具体来说，插入、删除和查找操作的平均时间复杂度都是O(logN)。这是因为红黑树保持着平衡性，使得树的高度不会超过logN，因此每次操作最多需要遍历树的高度次，即O(logN)次。而在最坏情况下，红黑树的操作时间复杂度也是O(logN)，这是因为红黑树的平衡性保证了树的高度不会超过2logN。因此，红黑树的时间复杂度是非常高效的，它可以应用于各种数据结构和算法中。


In [55]:
print(chain.run("人有哪八大系统"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '人有哪八大系统'}
> Finished chain.
？

这个问题涉及到人体的解剖学和生理学，其中包括：
1.消化系统：负责消化食物并吸收营养物质。
2.呼吸系统：负责吸入氧气并排出二氧化碳。
3.循环系统：负责输送血液和营养物质到身体各个部位。
4.泌尿系统：负责排泄废物和维持体液平衡。
5.神经系统：负责控制身体的运动和感觉，以及调节内部环境。
6.内分泌系统：负责产生和分泌激素，调节身体的生理过程。
7.免疫系统：负责保护身体免受疾病和感染。
8.生殖系统：负责生殖和产生后代。


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains